In [8]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Embedding, Dense, concatenate
from keras.layers import LSTM, Bidirectional, Dropout
from sklearn.model_selection import train_test_split

# Load data into a Pandas dataframe
df = pd.DataFrame({
    'num_col1': [1, 2, 3, 4,5,6,7,8],
    'num_col2': [10, 20, 30, 40,50,60,70,80],
    'cat_col1': ['cat1', 'cat2', 'cat3', 'cat4','cat5','cat6','cat7','cat8'],
    'cat_col2': ['A', 'B', 'C', 'D','A', 'B', 'C', 'D'],
    'text_col': ['the quick brown fox', 'jumps over the lazy dog', 'hello world', 'deep learning is cool','hello world', 'deep learning is cool','hello world', 'deep learning is cool'],
    'target': [0, 0, 1, 1,0, 0, 1, 1]
})

# Separate numerical, categorical, and text data
num_cols = ['num_col1', 'num_col2']
cat_cols = ['cat_col1', 'cat_col2']
text_col = 'text_col'

X_num = df[num_cols].values
X_cat = df[cat_cols].values
X_text = df[text_col].values

# Preprocess text data
# ...

# Split the data into training, validation, and test sets
X_train_num, X_val_num, X_train_cat, X_val_cat, X_train_text, X_val_text, y_train, y_val = train_test_split(X_num, X_cat, X_text, df['target'].values, test_size=0.2, random_state=42)
X_val_num, X_test_num, X_val_cat, X_test_cat, X_val_text, X_test_text, y_val, y_test = train_test_split(X_val_num, X_val_cat, X_val_text, y_val, test_size=0.1, random_state=42)

# Define the model
num_inputs = Input(shape=(len(num_cols),))
cat_inputs = Input(shape=(len(cat_cols),))
text_inputs = Input(shape=(30,))

embeddings = Embedding(input_dim=1000, output_dim=50, input_length=30)(text_inputs)
lstm = Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))(embeddings)

x = concatenate([num_inputs, cat_inputs, lstm])
x = Dense(units=64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=32, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(units=2, activation='softmax')(x)

model = Model(inputs=[num_inputs, cat_inputs, text_inputs], outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train_num, X_train_cat, X_train_text], y_train, epochs=10, batch_size=32, validation_data=([X_val_num, X_val_cat, X_val_text], y_val))

# Evaluate the model
loss, accuracy = model.evaluate([X_test_num, X_test_cat, X_test_text], y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


Epoch 1/10


ValueError: in user code:

    File "C:\Users\jessi\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\jessi\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\jessi\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\jessi\anaconda3\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\jessi\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\jessi\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 232, in assert_input_compatibility
        raise ValueError(

    ValueError: Exceptio